In [1]:
import pandas as pd
import os

# Importando tabelas

In [2]:
df_dados = pd.read_excel('dados.xlsx')
df_fator = pd.read_excel('fatores.xlsx')

lst_meses = ['Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov', 'Dez']
lst_meses_dados = ['Dez anterior','Jan', 'Fev', 'Mar', 'Abr', 'Mai', 'Jun', 'Jul', 'Ago', 'Set', 'Out', 'Nov']
lst_layout = ['Galpões', 'Complemento','Registro', 'Empresa', 'Tipo de imóvel']

# Gerando Data Frame das diferenças

In [3]:
df_diferença = df_dados[lst_layout]

for m, d in zip(lst_meses, lst_meses_dados):
    dif = df_dados.loc[:, m] - df_dados.loc[:, d]
    df_diferença.loc[:, m] = dif

df_diferença = df_diferença.loc[df_diferença['Complemento'] != 'Fundos', :].reset_index(drop=True)

df_fator.loc[3, 'Volume max de água m3'] = 999999

display(df_fator)

C:\Users\Gabriel\AppData\Local\Temp\ipykernel_7240\696336622.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diferença.loc[:, m] = dif
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_7240\696336622.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_diferença.loc[:, m] = dif
C:\Users\Gabriel\AppData\Local\Temp\ipykernel_7240\696336622.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cave

,Volume min de água m3,Volume max de água m3,Tipo de fator,Fator Industrial,Fator Residencial
0,0,10.0,Fixo,51.04,25.42
1,10,20.0,Multiplicador,6.05,3.54
2,20,50.0,Multiplicador,9.76,5.45
3,50,999999.0,Multiplicador,11.46,6.51


# Gerando Data Frame dos preços

In [4]:
# As funções definidas a seguir foram somente para organizar a funcionalidade do loop

# def df_taxas_xm(index, m, type):
#     df_temp = df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
#     return df_temp

# def df_taxas_Mxm(index, m, type):
#     #type foi utilizada fora da função como o tipo de imóvel. No caso do loop, como var
#     df_temp = df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]
#     return df_temp

# def df_taxas_Mx(index, m, type):
#     df_temp = df_preço.loc[(df_diferença[m] >= df_fator.loc[index,'Volume min de água m3']) & (df_diferença['Tipo de imóvel'] == type), m]
#     return df_temp


df_preço = df_diferença.loc[df_diferença['Galpões'] != 'Poço', lst_layout]

for m in lst_meses:
    
    for type in ['Industrial', 'Residencial']:
        df_preço.loc[(df_diferença[m] < df_fator.loc[0,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type), m] = df_fator.loc[0, f'Fator {type}']
        
        for index in range(1, 4):
            df_temp = df_diferença.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m]  
            df_preço.loc[(df_diferença[m] < df_fator.loc[index,'Volume max de água m3']) & (df_diferença['Tipo de imóvel'] == type) & (df_diferença[m] >= df_fator.loc[index, 'Volume min de água m3']), m] = df_temp * df_fator.loc[index, f'Fator {type}']

df_preço = df_preço.round(2)

# TOTAL
last_index = df_preço.index[-1]
df_preço.loc[last_index + 1] = df_preço.sum(numeric_only=True, axis=0)
df_preço.loc[last_index + 1, 'Galpões'] = 'TOTAL'

df_preço.to_excel('Tabela preços água.xlsx', index=False)

# display(df_preço[['Registro']+lst_meses])




        


# Separando arquivos na pasta compartilhamento

In [5]:
if not os.path.exists(r'compartilhamento'):
    os.mkdir(r'compartilhamento')


KeyError: 'Empresa'